In [1]:
import pandas as pd
import numpy as np

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

pd.set_option('display.max_columns', None)

In [2]:
anime = pd.read_csv('anime.csv')
anime.head()

,anime_id,title,type,score,scored_by,status,episodes,start_date,end_date,source,members,favorites,episode_duration,total_duration,rating,sfw,approved,created_at,updated_at,start_year,start_season,real_start_date,real_end_date,broadcast_day,broadcast_time,genres,themes,demographics,studios,producers,licensors,synopsis,background,main_picture,url,trailer_url,title_english,title_japanese,title_synonyms
0,5114,Fullmetal Alchemist: Brotherhood,tv,9.13,1871705,finished_airing,64.0,2009-04-05,2010-07-04,manga,2932347,204645,0 days 00:24:20,1 days 01:57:20,r,True,True,2008-08-21 03:35:22+00:00,2022-04-18 05:06:13+00:00,2009.0,spring,2009-04-05,2010-07-04,sunday,17:00:00,"['Action', 'Adventure', 'Drama', 'Fantasy']",['Military'],['Shounen'],['Bones'],"['Aniplex', 'Square Enix', 'Mainichi Broadcast...","['Funimation', 'Aniplex of America']",After a horrific alchemy experiment goes wrong...,NaN,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,https://www.youtube.com/watch?v=--IcmZkvL0Q,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,['Hagane no Renkinjutsushi: Fullmetal Alchemis...
1,11061,Hunter x Hunter (2011),tv,9.04,1509622,finished_airing,148.0,2011-10-02,2014-09-24,manga,2418883,185178,0 days 00:23:37,2 days 10:15:16,pg_13,True,True,2011-07-27 08:41:39+00:00,2022-04-18 05:11:21+00:00,2011.0,fall,2011-10-02,2014-09-24,sunday,10:55:00,"['Action', 'Adventure', 'Fantasy']",[],['Shounen'],['Madhouse'],"['VAP', 'Nippon Television Network', 'Shueisha']",['VIZ Media'],Hunters devote themselves to accomplishing haz...,NaN,https://cdn.myanimelist.net/images/anime/1337/...,https://myanimelist.net/anime/11061/Hunter_x_H...,https://www.youtube.com/watch?v=D9iTQRB4XRk,Hunter x Hunter,HUNTER×HUNTER（ハンター×ハンター）,['HxH (2011)']
2,38524,Shingeki no Kyojin Season 3 Part 2,tv,9.07,1329500,finished_airing,10.0,2019-04-29,2019-07-01,manga,1881734,51931,0 days 00:23:55,0 days 03:59:10,r,True,True,2018-10-14 18:13:45+00:00,2022-04-18 05:02:23+00:00,2019.0,spring,2019-04-29,2019-07-01,monday,00:10:00,"['Action', 'Drama']","['Gore', 'Military', 'Survival']",['Shounen'],['Wit Studio'],"['Production I.G', 'Dentsu', 'Mainichi Broadca...",['Funimation'],Seeking to restore humanity's diminishing hope...,Shingeki no Kyojin adapts content from volumes...,https://cdn.myanimelist.net/images/anime/1517/...,https://myanimelist.net/anime/38524/Shingeki_n...,https://www.youtube.com/watch?v=hKHepjfj5Tw,Attack on Titan Season 3 Part 2,進撃の巨人 Season3 Part.2,[]
3,9253,Steins;Gate,tv,9.08,1252286,finished_airing,24.0,2011-04-06,2011-09-14,visual_novel,2269121,173088,0 days 00:24:20,0 days 09:44:00,pg_13,True,True,2010-07-26 09:23:40+00:00,2022-05-01 21:24:42+00:00,2011.0,spring,2011-04-06,2011-09-14,wednesday,02:05:00,"['Drama', 'Sci-Fi', 'Suspense']","['Psychological', 'Time Travel']",[],['White Fox'],"['Frontier Works', 'Media Factory', 'Movic', '...",['Funimation'],Eccentric scientist Rintarou Okabe has a never...,Steins;Gate is based on 5pb. and Nitroplus' re...,https://cdn.myanimelist.net/images/anime/5/731...,https://myanimelist.net/anime/9253/Steins_Gate,https://www.youtube.com/watch?v=27OZc-ku6is,Steins;Gate,STEINS;GATE,[]
4,28851,Koe no Katachi,movie,8.95,1398608,finished_airing,1.0,2016-09-17,2016-09-17,manga,2001335,77431,0 days 02:10:03,0 days 02:10:03,pg_13,True,True,2014-12-15 08:00:46+00:00,2022-04-18 04:55:56+00:00,2016.0,summer,2016-09-17,2016-09-17,NaN,NaN,['Drama'],['Romantic Subtext'],['Shounen'],['Kyoto Animation'],"['Shochiku', 'Pony Canyon', 'Kodansha', 'ABC A...","['Eleven Arts', 'NYAV Post']","As a wild youth, elementary school student Sho...",Winner of the Excellence Award on the 20th Jap...,https://cdn.myanimelist.net/images/anime/1122/...,https://myanimelist.net/anime/28851/Koe_no_Kat...,https://www.youtube.com/watch?v=XBNWo25izJ8,A Silent Voice,聲の形,['The Shape of Voice']


In [3]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24165 entries, 0 to 24164
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   anime_id          24165 non-null  int64  
 1   title             24165 non-null  object 
 2   type              24092 non-null  object 
 3   score             14272 non-null  float64
 4   scored_by         24165 non-null  int64  
 5   status            24165 non-null  object 
 6   episodes          23397 non-null  float64
 7   start_date        21391 non-null  object 
 8   end_date          19912 non-null  object 
 9   source            20404 non-null  object 
 10  members           24165 non-null  int64  
 11  favorites         24165 non-null  int64  
 12  episode_duration  23302 non-null  object 
 13  total_duration    23002 non-null  object 
 14  rating            23148 non-null  object 
 15  sfw               24165 non-null  bool   
 16  approved          24165 non-null  bool  

In [4]:
anime.dropna(subset=['title', 'score', 'rating'], inplace=True)

Menghitung score

In [5]:
C = anime.score.mean()
C

6.461685948081264

In [6]:
m = anime['scored_by'].quantile(0.75)
m

11875.75

In [7]:
anime = anime.loc[anime['scored_by'] >= m]
anime.shape

(3544, 39)

In [8]:
def weighted_rating(x, m=m, C=C):
    v = x['scored_by']
    R = x['score']
    # Calculation based on the IMDB formula
    return np.round(((v/(v+m) * R) + (m/(m+v) * C)), 2)

In [9]:
anime['score_'] = anime.apply(weighted_rating, axis=1)

In [10]:
anime.drop(['score', 'scored_by', 'members', 'favorites', 'broadcast_day', 'broadcast_time',
            'created_at', 'updated_at'], axis=1, inplace=True)

In [11]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3544 entries, 0 to 8743
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   anime_id          3544 non-null   int64  
 1   title             3544 non-null   object 
 2   type              3544 non-null   object 
 3   status            3544 non-null   object 
 4   episodes          3533 non-null   float64
 5   start_date        3544 non-null   object 
 6   end_date          3521 non-null   object 
 7   source            3523 non-null   object 
 8   episode_duration  3544 non-null   object 
 9   total_duration    3533 non-null   object 
 10  rating            3544 non-null   object 
 11  sfw               3544 non-null   bool   
 12  approved          3544 non-null   bool   
 13  start_year        3542 non-null   float64
 14  start_season      3542 non-null   object 
 15  real_start_date   3544 non-null   object 
 16  real_end_date     3521 non-null   object 


In [12]:
list_col = ['genres', 'themes', 'demographics', 'studios', 'producers', 'licensors', 'title_synonyms']
mapping = [("[", ""), ("]", ""), ("'", ""), ('"', '')]
for i, j in mapping:
    for k in list_col:
        anime[k] = anime[k].apply(lambda x: x.replace(i, j))

In [13]:
anime

,anime_id,title,type,status,episodes,start_date,end_date,source,episode_duration,total_duration,rating,sfw,approved,start_year,start_season,real_start_date,real_end_date,genres,themes,demographics,studios,producers,licensors,synopsis,background,main_picture,url,trailer_url,title_english,title_japanese,title_synonyms,score_
0,5114,Fullmetal Alchemist: Brotherhood,tv,finished_airing,64.0,2009-04-05,2010-07-04,manga,0 days 00:24:20,1 days 01:57:20,r,True,True,2009.0,spring,2009-04-05,2010-07-04,"Action, Adventure, Drama, Fantasy",Military,Shounen,Bones,"Aniplex, Square Enix, Mainichi Broadcasting Sy...","Funimation, Aniplex of America",After a horrific alchemy experiment goes wrong...,NaN,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,https://www.youtube.com/watch?v=--IcmZkvL0Q,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",9.11
1,11061,Hunter x Hunter (2011),tv,finished_airing,148.0,2011-10-02,2014-09-24,manga,0 days 00:23:37,2 days 10:15:16,pg_13,True,True,2011.0,fall,2011-10-02,2014-09-24,"Action, Adventure, Fantasy",,Shounen,Madhouse,"VAP, Nippon Television Network, Shueisha",VIZ Media,Hunters devote themselves to accomplishing haz...,NaN,https://cdn.myanimelist.net/images/anime/1337/...,https://myanimelist.net/anime/11061/Hunter_x_H...,https://www.youtube.com/watch?v=D9iTQRB4XRk,Hunter x Hunter,HUNTER×HUNTER（ハンター×ハンター）,HxH (2011),9.02
2,38524,Shingeki no Kyojin Season 3 Part 2,tv,finished_airing,10.0,2019-04-29,2019-07-01,manga,0 days 00:23:55,0 days 03:59:10,r,True,True,2019.0,spring,2019-04-29,2019-07-01,"Action, Drama","Gore, Military, Survival",Shounen,Wit Studio,"Production I.G, Dentsu, Mainichi Broadcasting ...",Funimation,Seeking to restore humanity's diminishing hope...,Shingeki no Kyojin adapts content from volumes...,https://cdn.myanimelist.net/images/anime/1517/...,https://myanimelist.net/anime/38524/Shingeki_n...,https://www.youtube.com/watch?v=hKHepjfj5Tw,Attack on Titan Season 3 Part 2,進撃の巨人 Season3 Part.2,,9.05
3,9253,Steins;Gate,tv,finished_airing,24.0,2011-04-06,2011-09-14,visual_novel,0 days 00:24:20,0 days 09:44:00,pg_13,True,True,2011.0,spring,2011-04-06,2011-09-14,"Drama, Sci-Fi, Suspense","Psychological, Time Travel",,White Fox,"Frontier Works, Media Factory, Movic, AT-X, Ka...",Funimation,Eccentric scientist Rintarou Okabe has a never...,Steins;Gate is based on 5pb. and Nitroplus' re...,https://cdn.myanimelist.net/images/anime/5/731...,https://myanimelist.net/anime/9253/Steins_Gate,https://www.youtube.com/watch?v=27OZc-ku6is,Steins;Gate,STEINS;GATE,,9.06
4,28851,Koe no Katachi,movie,finished_airing,1.0,2016-09-17,2016-09-17,manga,0 days 02:10:03,0 days 02:10:03,pg_13,True,True,2016.0,summer,2016-09-17,2016-09-17,Drama,Romantic Subtext,Shounen,Kyoto Animation,"Shochiku, Pony Canyon, Kodansha, ABC Animation...","Eleven Arts, NYAV Post","As a wild youth, elementary school student Sho...",Winner of the Excellence Award on the 20th Jap...,https://cdn.myanimelist.net/images/anime/1122/...,https://myanimelist.net/anime/28851/Koe_no_Kat...,https://www.youtube.com/watch?v=XBNWo25izJ8,A Silent Voice,聲の形,The Shape of Voice,8.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8445,40074,Gibiate,tv,finished_airing,12.0,2020-07-15,2020-09-30,original,0 days 00:24:00,0 days 04:48:00,r,True,True,2020.0,summer,2020-07-15,2020-09-30,"Action, Fantasy, Horror","Martial Arts, Samurai",,"Studio elle, l-a-unch・BOX","Nexon, Age Global Networks, Forecast Communica...",,The story takes place in Japan. The year is 20...,"Gibiate was created by GIBIATE Project, a glob...",https://cdn.myanimelist.net/images/anime/1035/...,https://myanimelist.net/anime/40074/Gibiate,https://www.youtube.com/watch?v=N_sO4mcdWxM,NaN,ジビエート,,4.75
8514,34823,Souryo to Majiwaru Shikiyoku no Yoru ni...,ona,finished_airing,12.0,2017-04-03,2017-06-19,manga,0 d

In [14]:
anime.rating.value_counts()

pg_13    2291
r         666
r+        328
g         108
pg        105
rx         46
Name: rating, dtype: int64

In [15]:
anime_soup = anime[['title', 'rating','genres', 'themes', 'demographics', 'studios']]

In [16]:
anime_soup

,title,rating,genres,themes,demographics,studios
0,Fullmetal Alchemist: Brotherhood,r,"Action, Adventure, Drama, Fantasy",Military,Shounen,Bones
1,Hunter x Hunter (2011),pg_13,"Action, Adventure, Fantasy",,Shounen,Madhouse
2,Shingeki no Kyojin Season 3 Part 2,r,"Action, Drama","Gore, Military, Survival",Shounen,Wit Studio
3,Steins;Gate,pg_13,"Drama, Sci-Fi, Suspense","Psychological, Time Travel",,White Fox
4,Koe no Katachi,pg_13,Drama,Romantic Subtext,Shounen,Kyoto Animation
...,...,...,...,...,...,...
8445,Gibiate,r,"Action, Fantasy, Horror","Martial Arts, Samurai",,"Studio elle, l-a-unch・BOX"
8514,Souryo to Majiwaru Shikiyoku no Yoru ni...,rx,"Drama, Hentai, Romance",,,Seven
8661,Eiken: Eikenbu yori Ai wo Komete,pg_13,"Comedy, Ecchi","Harem, School",,J.C.Staff
8735,Shitcom,pg_13,"Comedy, Romance",,,


In [17]:
mapping_soup = [(" ", ""), (",", " ")]
for i, j in mapping_soup:
    for k in ['genres', 'themes', 'demographics', 'studios']:
        anime_soup[k] = anime_soup[k].apply(lambda x: x.replace(i, j))

In [18]:
anime_soup['soup'] = anime_soup.title + ' ' + anime_soup.rating + ' ' + anime_soup.genres + ' ' + anime_soup.themes + ' ' + anime_soup.demographics + ' ' + anime_soup.studios

In [19]:
anime_soup = anime_soup.soup

In [20]:
for i in anime_soup:
     anime_soup.loc[anime_soup==i] = " ".join(i.split())

In [21]:
anime_soup = anime_soup.apply(lambda x: x.lower())

In [22]:
anime_soup

0       fullmetal alchemist: brotherhood r action adve...
1       hunter x hunter (2011) pg_13 action adventure ...
2       shingeki no kyojin season 3 part 2 r action dr...
3       steins;gate pg_13 drama sci-fi suspense psycho...
4       koe no katachi pg_13 drama romanticsubtext sho...
                              ...                        
8445    gibiate r action fantasy horror martialarts sa...
8514    souryo to majiwaru shikiyoku no yoru ni... rx ...
8661    eiken: eikenbu yori ai wo komete pg_13 comedy ...
8735                         shitcom pg_13 comedy romance
8743              utsu musume sayuri r+ avantgarde comedy
Name: soup, Length: 3544, dtype: object

In [23]:
anime_soup.to_csv('anime_soup_genres_based.csv', index=False)

In [24]:
anime

,anime_id,title,type,status,episodes,start_date,end_date,source,episode_duration,total_duration,rating,sfw,approved,start_year,start_season,real_start_date,real_end_date,genres,themes,demographics,studios,producers,licensors,synopsis,background,main_picture,url,trailer_url,title_english,title_japanese,title_synonyms,score_
0,5114,Fullmetal Alchemist: Brotherhood,tv,finished_airing,64.0,2009-04-05,2010-07-04,manga,0 days 00:24:20,1 days 01:57:20,r,True,True,2009.0,spring,2009-04-05,2010-07-04,"Action, Adventure, Drama, Fantasy",Military,Shounen,Bones,"Aniplex, Square Enix, Mainichi Broadcasting Sy...","Funimation, Aniplex of America",After a horrific alchemy experiment goes wrong...,NaN,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,https://www.youtube.com/watch?v=--IcmZkvL0Q,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",9.11
1,11061,Hunter x Hunter (2011),tv,finished_airing,148.0,2011-10-02,2014-09-24,manga,0 days 00:23:37,2 days 10:15:16,pg_13,True,True,2011.0,fall,2011-10-02,2014-09-24,"Action, Adventure, Fantasy",,Shounen,Madhouse,"VAP, Nippon Television Network, Shueisha",VIZ Media,Hunters devote themselves to accomplishing haz...,NaN,https://cdn.myanimelist.net/images/anime/1337/...,https://myanimelist.net/anime/11061/Hunter_x_H...,https://www.youtube.com/watch?v=D9iTQRB4XRk,Hunter x Hunter,HUNTER×HUNTER（ハンター×ハンター）,HxH (2011),9.02
2,38524,Shingeki no Kyojin Season 3 Part 2,tv,finished_airing,10.0,2019-04-29,2019-07-01,manga,0 days 00:23:55,0 days 03:59:10,r,True,True,2019.0,spring,2019-04-29,2019-07-01,"Action, Drama","Gore, Military, Survival",Shounen,Wit Studio,"Production I.G, Dentsu, Mainichi Broadcasting ...",Funimation,Seeking to restore humanity's diminishing hope...,Shingeki no Kyojin adapts content from volumes...,https://cdn.myanimelist.net/images/anime/1517/...,https://myanimelist.net/anime/38524/Shingeki_n...,https://www.youtube.com/watch?v=hKHepjfj5Tw,Attack on Titan Season 3 Part 2,進撃の巨人 Season3 Part.2,,9.05
3,9253,Steins;Gate,tv,finished_airing,24.0,2011-04-06,2011-09-14,visual_novel,0 days 00:24:20,0 days 09:44:00,pg_13,True,True,2011.0,spring,2011-04-06,2011-09-14,"Drama, Sci-Fi, Suspense","Psychological, Time Travel",,White Fox,"Frontier Works, Media Factory, Movic, AT-X, Ka...",Funimation,Eccentric scientist Rintarou Okabe has a never...,Steins;Gate is based on 5pb. and Nitroplus' re...,https://cdn.myanimelist.net/images/anime/5/731...,https://myanimelist.net/anime/9253/Steins_Gate,https://www.youtube.com/watch?v=27OZc-ku6is,Steins;Gate,STEINS;GATE,,9.06
4,28851,Koe no Katachi,movie,finished_airing,1.0,2016-09-17,2016-09-17,manga,0 days 02:10:03,0 days 02:10:03,pg_13,True,True,2016.0,summer,2016-09-17,2016-09-17,Drama,Romantic Subtext,Shounen,Kyoto Animation,"Shochiku, Pony Canyon, Kodansha, ABC Animation...","Eleven Arts, NYAV Post","As a wild youth, elementary school student Sho...",Winner of the Excellence Award on the 20th Jap...,https://cdn.myanimelist.net/images/anime/1122/...,https://myanimelist.net/anime/28851/Koe_no_Kat...,https://www.youtube.com/watch?v=XBNWo25izJ8,A Silent Voice,聲の形,The Shape of Voice,8.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8445,40074,Gibiate,tv,finished_airing,12.0,2020-07-15,2020-09-30,original,0 days 00:24:00,0 days 04:48:00,r,True,True,2020.0,summer,2020-07-15,2020-09-30,"Action, Fantasy, Horror","Martial Arts, Samurai",,"Studio elle, l-a-unch・BOX","Nexon, Age Global Networks, Forecast Communica...",,The story takes place in Japan. The year is 20...,"Gibiate was created by GIBIATE Project, a glob...",https://cdn.myanimelist.net/images/anime/1035/...,https://myanimelist.net/anime/40074/Gibiate,https://www.youtube.com/watch?v=N_sO4mcdWxM,NaN,ジビエート,,4.75
8514,34823,Souryo to Majiwaru Shikiyoku no Yoru ni...,ona,finished_airing,12.0,2017-04-03,2017-06-19,manga,0 d

In [25]:
from datetime import timedelta

def get_time_hh_mm_ss(time_delta):
    sec = pd.Timedelta(time_delta).total_seconds()
    
    td_str = str(timedelta(seconds=sec))

    x = td_str.split(':')
    if x[0] == '0':
        return f'{x[1]} min. {x[2]} sec.'
    else:
        return f'{x[0]} hr. {x[1]} min. {x[2]} sec.'

In [26]:
anime['duration'] = anime['episode_duration'].apply(get_time_hh_mm_ss)

In [27]:
anime.head()

,anime_id,title,type,status,episodes,start_date,end_date,source,episode_duration,total_duration,rating,sfw,approved,start_year,start_season,real_start_date,real_end_date,genres,themes,demographics,studios,producers,licensors,synopsis,background,main_picture,url,trailer_url,title_english,title_japanese,title_synonyms,score_,duration
0,5114,Fullmetal Alchemist: Brotherhood,tv,finished_airing,64.0,2009-04-05,2010-07-04,manga,0 days 00:24:20,1 days 01:57:20,r,True,True,2009.0,spring,2009-04-05,2010-07-04,"Action, Adventure, Drama, Fantasy",Military,Shounen,Bones,"Aniplex, Square Enix, Mainichi Broadcasting Sy...","Funimation, Aniplex of America",After a horrific alchemy experiment goes wrong...,NaN,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,https://www.youtube.com/watch?v=--IcmZkvL0Q,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",9.11,24 min. 20 sec.
1,11061,Hunter x Hunter (2011),tv,finished_airing,148.0,2011-10-02,2014-09-24,manga,0 days 00:23:37,2 days 10:15:16,pg_13,True,True,2011.0,fall,2011-10-02,2014-09-24,"Action, Adventure, Fantasy",,Shounen,Madhouse,"VAP, Nippon Television Network, Shueisha",VIZ Media,Hunters devote themselves to accomplishing haz...,NaN,https://cdn.myanimelist.net/images/anime/1337/...,https://myanimelist.net/anime/11061/Hunter_x_H...,https://www.youtube.com/watch?v=D9iTQRB4XRk,Hunter x Hunter,HUNTER×HUNTER（ハンター×ハンター）,HxH (2011),9.02,23 min. 37 sec.
2,38524,Shingeki no Kyojin Season 3 Part 2,tv,finished_airing,10.0,2019-04-29,2019-07-01,manga,0 days 00:23:55,0 days 03:59:10,r,True,True,2019.0,spring,2019-04-29,2019-07-01,"Action, Drama","Gore, Military, Survival",Shounen,Wit Studio,"Production I.G, Dentsu, Mainichi Broadcasting ...",Funimation,Seeking to restore humanity's diminishing hope...,Shingeki no Kyojin adapts content from volumes...,https://cdn.myanimelist.net/images/anime/1517/...,https://myanimelist.net/anime/38524/Shingeki_n...,https://www.youtube.com/watch?v=hKHepjfj5Tw,Attack on Titan Season 3 Part 2,進撃の巨人 Season3 Part.2,,9.05,23 min. 55 sec.
3,9253,Steins;Gate,tv,finished_airing,24.0,2011-04-06,2011-09-14,visual_novel,0 days 00:24:20,0 days 09:44:00,pg_13,True,True,2011.0,spring,2011-04-06,2011-09-14,"Drama, Sci-Fi, Suspense","Psychological, Time Travel",,White Fox,"Frontier Works, Media Factory, Movic, AT-X, Ka...",Funimation,Eccentric scientist Rintarou Okabe has a never...,Steins;Gate is based on 5pb. and Nitroplus' re...,https://cdn.myanimelist.net/images/anime/5/731...,https://myanimelist.net/anime/9253/Steins_Gate,https://www.youtube.com/watch?v=27OZc-ku6is,Steins;Gate,STEINS;GATE,,9.06,24 min. 20 sec.
4,28851,Koe no Katachi,movie,finished_airing,1.0,2016-09-17,2016-09-17,manga,0 days 02:10:03,0 days 02:10:03,pg_13,True,True,2016.0,summer,2016-09-17,2016-09-17,Drama,Romantic Subtext,Shounen,Kyoto Animation,"Shochiku, Pony Canyon, Kodansha, ABC Animation...","Eleven Arts, NYAV Post","As a wild youth, elementary school student Sho...",Winner of the Excellence Award on the 20th Jap...,https://cdn.myanimelist.net/images/anime/1122/...,https://myanimelist.net/anime/28851/Koe_no_Kat...,https://www.youtube.com/watch?v=XBNWo25izJ8,A Silent Voice,聲の形,The Shape of Voice,8.93,2 hr. 10 min. 03 sec.


In [28]:
anime.isna().sum()

anime_id               0
title                  0
type                   0
status                 0
episodes              11
start_date             0
end_date              23
source                21
episode_duration       0
total_duration        11
rating                 0
sfw                    0
approved               0
start_year             2
start_season           2
real_start_date        0
real_end_date         23
genres                 0
themes                 0
demographics           0
studios                0
producers              0
licensors              0
synopsis               4
background          2614
main_picture           0
url                    0
trailer_url         1194
title_english        759
title_japanese         1
title_synonyms         0
score_                 0
duration               0
dtype: int64

In [29]:
anime.rating.value_counts()

pg_13    2291
r         666
r+        328
g         108
pg        105
rx         46
Name: rating, dtype: int64

In [30]:
rating_mapping = [('pg_13', 'PG-13 - Teens 13 or older'), 
                  ('r', 'R - 17+ (violence & profanity)'), 
                  ('r+', 'R+ - Mild Nudity'), 
                  ('g', 'G - All Ages'), 
                  ('pg', 'PG - Children'), 
                  ('rx', 'Rx - Hentai')]

for i, j in rating_mapping:
    anime['rating'] = anime['rating'].replace([i],j)

In [31]:
anime.rating.value_counts()

PG-13 - Teens 13 or older         2291
R - 17+ (violence & profanity)     666
R+ - Mild Nudity                   328
G - All Ages                       108
PG - Children                      105
Rx - Hentai                         46
Name: rating, dtype: int64

In [32]:
anime['source'] = anime['source'].fillna('Unknown')
anime['source'] = anime['source'].apply(lambda x: x.replace('_',' '))

In [33]:
anime['status'] = anime['status'].apply(lambda x: x.replace('_', ' '))

In [34]:
anime.drop(['anime_id', 'url'], axis=1, inplace=True)

In [35]:
list_float = ['episodes', 'start_year']
anime[list_float] = anime[list_float].apply(lambda x: x.fillna(-1))
anime[list_float]=anime[list_float].astype('int')

In [36]:
anime[list_float] = anime[list_float].apply(lambda x: x.replace(-1, 'Unknown'))

anime['start_season'] = anime['start_season'].fillna('Unknown')


anime['end_date'] = anime['end_date'].fillna('?')

list_unknown_ = ['genres', 'themes', 'demographics', 'studios', 'producers', 'licensors']
anime[list_unknown_] = anime[list_unknown_].apply(lambda x: x.replace('', 'Unknown'))

anime['synopsis'] = anime['synopsis'].fillna('There no synopsis for this title')

list_ =['background','title_english', 'title_japanese']
anime[list_] = anime[list_].fillna('-')

anime['trailer?'] = anime.trailer_url.isna()

In [37]:
anime.isna().sum()

title                  0
type                   0
status                 0
episodes               0
start_date             0
end_date               0
source                 0
episode_duration       0
total_duration        11
rating                 0
sfw                    0
approved               0
start_year             0
start_season           0
real_start_date        0
real_end_date         23
genres                 0
themes                 0
demographics           0
studios                0
producers              0
licensors              0
synopsis               0
background             0
main_picture           0
trailer_url         1194
title_english          0
title_japanese         0
title_synonyms         0
score_                 0
duration               0
trailer?               0
dtype: int64

In [38]:
anime.to_csv('anime_clean.csv', index=False)

In [39]:
anime

,title,type,status,episodes,start_date,end_date,source,episode_duration,total_duration,rating,sfw,approved,start_year,start_season,real_start_date,real_end_date,genres,themes,demographics,studios,producers,licensors,synopsis,background,main_picture,trailer_url,title_english,title_japanese,title_synonyms,score_,duration,trailer?
0,Fullmetal Alchemist: Brotherhood,tv,finished airing,64,2009-04-05,2010-07-04,manga,0 days 00:24:20,1 days 01:57:20,R - 17+ (violence & profanity),True,True,2009,spring,2009-04-05,2010-07-04,"Action, Adventure, Drama, Fantasy",Military,Shounen,Bones,"Aniplex, Square Enix, Mainichi Broadcasting Sy...","Funimation, Aniplex of America",After a horrific alchemy experiment goes wrong...,-,https://cdn.myanimelist.net/images/anime/1223/...,https://www.youtube.com/watch?v=--IcmZkvL0Q,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",9.11,24 min. 20 sec.,False
1,Hunter x Hunter (2011),tv,finished airing,148,2011-10-02,2014-09-24,manga,0 days 00:23:37,2 days 10:15:16,PG-13 - Teens 13 or older,True,True,2011,fall,2011-10-02,2014-09-24,"Action, Adventure, Fantasy",Unknown,Shounen,Madhouse,"VAP, Nippon Television Network, Shueisha",VIZ Media,Hunters devote themselves to accomplishing haz...,-,https://cdn.myanimelist.net/images/anime/1337/...,https://www.youtube.com/watch?v=D9iTQRB4XRk,Hunter x Hunter,HUNTER×HUNTER（ハンター×ハンター）,HxH (2011),9.02,23 min. 37 sec.,False
2,Shingeki no Kyojin Season 3 Part 2,tv,finished airing,10,2019-04-29,2019-07-01,manga,0 days 00:23:55,0 days 03:59:10,R - 17+ (violence & profanity),True,True,2019,spring,2019-04-29,2019-07-01,"Action, Drama","Gore, Military, Survival",Shounen,Wit Studio,"Production I.G, Dentsu, Mainichi Broadcasting ...",Funimation,Seeking to restore humanity's diminishing hope...,Shingeki no Kyojin adapts content from volumes...,https://cdn.myanimelist.net/images/anime/1517/...,https://www.youtube.com/watch?v=hKHepjfj5Tw,Attack on Titan Season 3 Part 2,進撃の巨人 Season3 Part.2,,9.05,23 min. 55 sec.,False
3,Steins;Gate,tv,finished airing,24,2011-04-06,2011-09-14,visual novel,0 days 00:24:20,0 days 09:44:00,PG-13 - Teens 13 or older,True,True,2011,spring,2011-04-06,2011-09-14,"Drama, Sci-Fi, Suspense","Psychological, Time Travel",Unknown,White Fox,"Frontier Works, Media Factory, Movic, AT-X, Ka...",Funimation,Eccentric scientist Rintarou Okabe has a never...,Steins;Gate is based on 5pb. and Nitroplus' re...,https://cdn.myanimelist.net/images/anime/5/731...,https://www.youtube.com/watch?v=27OZc-ku6is,Steins;Gate,STEINS;GATE,,9.06,24 min. 20 sec.,False
4,Koe no Katachi,movie,finished airing,1,2016-09-17,2016-09-17,manga,0 days 02:10:03,0 days 02:10:03,PG-13 - Teens 13 or older,True,True,2016,summer,2016-09-17,2016-09-17,Drama,Romantic Subtext,Shounen,Kyoto Animation,"Shochiku, Pony Canyon, Kodansha, ABC Animation...","Eleven Arts, NYAV Post","As a wild youth, elementary school student Sho...",Winner of the Excellence Award on the 20th Jap...,https://cdn.myanimelist.net/images/anime/1122/...,https://www.youtube.com/watch?v=XBNWo25izJ8,A Silent Voice,聲の形,The Shape of Voice,8.93,2 hr. 10 min. 03 sec.,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8445,Gibiate,tv,finished airing,12,2020-07-15,2020-09-30,original,0 days 00:24:00,0 days 04:48:00,R - 17+ (violence & profanity),True,True,2020,summer,2020-07-15,2020-09-30,"Action, Fantasy, Horror","Martial Arts, Samurai",Unknown,"Studio elle, l-a-unch・BOX","Nexon, Age Global Networks, Forecast Communica...",Unknown,The story takes place in Japan. The year is 20...,"Gibiate was created by GIBIATE Project, a glob...",https://cdn.myanimelist.net/images/anime/1035/...,https://www.youtube.com/watch?v=N_sO4mcdWxM,-,ジビエート,,4.75,24 min. 00 sec.,False
8514,Souryo to Majiwaru Shikiyoku no Yoru ni...,ona,finished airing,12,2017-04-03,2017-06-19,manga,0 days 00:04:09,0 days 00:49:48,Rx - Hentai,Fals